<a href="https://colab.research.google.com/github/purva1803/Prepinsta_Internship/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
url = 'http://books.toscrape.com/'
response = requests.get(url)

In [ ]:
response

<Response [200]>

In [ ]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [ ]:
soup = bs(response.text)

In [ ]:
type(soup)

bs4.BeautifulSoup

To inspect all the HTML tag and to identify the book tag so that we can extract information about the books.

In [ ]:
books_tag = soup.find_all('article', class_='product_pod')

To select a single book and extract all the information we **can**

In [ ]:
book_tag = books_tag[0]
book_tag

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

To collect book from title attribute

In [ ]:
title_tag = book_tag.find('a', title=True)
title_tag

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

To get the book tilte

In [ ]:
title_tag['title']

'A Light in the Attic'

Follow the same process (as above) to extract ratings, price & book_link.

Rating:

In [ ]:
rating = book_tag.find('p')['class'][1]
rating

'Three'

Price:

In [ ]:
price = book_tag.find('p', class_='price_color').text[1:]
price

'£51.77'

Book link:

In [ ]:
link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
link

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [ ]:
def get_details(book_tag):
    title = book_tag.find('a', title=True)['title']
    rating = book_tag.find('p')['class'][1]
    price = book_tag.find('p', class_='price_color').text[1:]
    link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    return title, rating, price, link

In [ ]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:
        return bs(resp.text)
    else: return None


def get_books(url):
    """Extact details from all the book tags"""
    soup = get_soup(url)
    book_tags = soup.find_all('article', class_='product_pod')

    books = []
    for book_tag in book_tags:
        books.append(get_details(book_tag))

    return books

In [ ]:
url = 'http://books.toscrape.com/'
books = get_books(url)
len(books)

20

In [ ]:
books[:3]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('Tipping the Velvet',
  'One',
  '£53.74',
  'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'),
 ('Soumission',
  'One',
  '£50.10',
  'http://books.toscrape.com/catalogue/soumission_998/index.html')]

In [ ]:
import pandas as pd

def get_all_books(page = 3):
    books = []
    for i in range(1, page+1):
        ## This is how the url changes with every page
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            book_tags = soup.find_all('article', class_='product_pod')

            for book_tag in book_tags:
                books.append(get_details(book_tag))

    books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
    return books

In [ ]:
df = get_all_books(3)
df.head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

def get_soup(url):
    """Takes URL and returns a soup object"""
    try:
        resp = requests.get(url)
        resp.raise_for_status()  # Raise HTTPError for bad status codes
        return bs(resp.text, 'html.parser')
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None


def get_details(book_tag):
    """Extract title, rating, price, and link from a book tag"""
    title = book_tag.find('a', title=True)['title'] if book_tag.find('a', title=True) else None

    rating_tag = book_tag.find('p', class_='star-rating')
    rating = rating_tag.attrs['class'][1] if rating_tag else None

    price_tag = book_tag.find('p', class_='price_color')
    price = price_tag.text[1:] if price_tag else None

    link_tag = book_tag.find('a')
    link = 'http://books.toscrape.com/' + link_tag['href'] if link_tag else None

    return title, rating, price, link


def get_all_books(page=3):
    """Scrape book information from multiple pages"""
    books = []
    for i in range(1, page+1):
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            try:
                book_tags = soup.find_all('article', class_='product_pod')
                for book_tag in book_tags:
                    books.append(get_details(book_tag))
            except Exception as e:
                print(f'Error processing page {i}: {e}')

            time.sleep(5)  # Sleep before making the next request

    return pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])

# Example usage:
books_df = get_all_books(page=3)
print(books_df)


                                                title rating   price  \
0                                A Light in the Attic  Three  £51.77   
1                                  Tipping the Velvet    One  £53.74   
2                                          Soumission    One  £50.10   
3                                       Sharp Objects   Four  £47.82   
4               Sapiens: A Brief History of Humankind   Five  £54.23   
5                                     The Requiem Red    One  £22.65   
6   The Dirty Little Secrets of Getting Your Dream...   Four  £33.34   
7   The Coming Woman: A Novel Based on the Life of...  Three  £17.93   
8   The Boys in the Boat: Nine Americans and Their...   Four  £22.60   
9                                     The Black Maria    One  £52.15   
10     Starving Hearts (Triangular Trade Trilogy, #1)    Two  £13.99   
11                              Shakespeare's Sonnets   Four  £20.66   
12                                        Set Me Free   Five  £1